 # pttnet

 ## 1.  Network Structure

 - Node
   - Each node is a **user** (identified by ID)
   - Each node has a "corpus", consisting of comments
       - Comments are grouped under dates (yyyy-mm-dd)
 - Edge
   - If two user both left comments under a post, an edge is formed between them.
   - An edge is essentially **a post** and contains the following attributes (aside the nodes):

       ```json
       {
           "title": "她答應了!她答應了!",
           "isRe": 0,
           "tag": "分享",
           "date": "2006-12-24",
           "board": "Boy-Girl",
           "opinion": "pos-pos",
           "text": "20061224_2348_M.1166975303.A.F56.json"
       }
       ```

## 2. High-level API: creating undirected graph

### 2.1 基本步驟

1. 使用 `MultiGraph()` 建立一個 [`networkx.MultiGraph()`](https://networkx.github.io/documentation/stable/reference/classes/multigraph.html)
    - edge 是 **undirected** & **unweighted**
    - 每對 node 之間可以有 **1 個以上的 edge**

1. 透過 `Graph()` 將 `MultiGraph()` 的網絡**縮減**
    - 每對 node 之間**最多只有 1 個 edge**
    - edge 是 **undirected** & ***weighted***

### 2.2 一個例子

在這個例子中，我先找出 2007-2008 年於 Boy-Girl 板上與 `airlow` 在相同文章留過言的所有使用者，並把結果儲存於一個 MultiGraph 裡：

In [22]:
from pttnet.graph import MultiGraph, Graph

conditions = {
    'nodes': {'airlow'}
}
MG = MultiGraph(edge_condition=conditions, boards=['Boy-Girl'], years=[2007, 2008])

In [23]:
print(f"Num of nodes: {len(MG.nodes)}")
print(f"Num of edges: {len(MG.edges)}")

Num of nodes: 1865
Num of edges: 37224


`MG.nodes` 裡面是這個 graph 裡面的所有節點，每個節點是一個 [`pttnet.graph.Node`](https://yongfu.name/ptt-network/html/doc/Node.html) 物件，具有 `id` 與 `corpus` 這些 attributes。

為了方便取得這些 Node 的資訊，可以將 `MG.nodes` 裡的 Nodes 另外建立一個 dictionary，這樣就可直接使用某個 Node 的 `id` 取得資料： 

In [24]:
nodes = {n.id: n for n in MG.nodes}
nodes.get('airlow')

<Node, node_id: airlow>

nodes.get('airlow').id

In [20]:
nodes.get('airlow').corpus

{'2019-10-21': [{'type': 'neu',
   'content': '有\u3000些人\u3000再\u3000怎麼\u3000努力\u3000，\u3000個\u3000人\u3000特質\u3000就是\u3000不\u3000討喜\u3000，\u3000就\u3000像\u3000不\u3000受\u3000歡迎\u3000的諧',
   'board': 'Boy-Girl',
   'ord': 20,
   'src': '20191021_0204_M.1571594694.A.D9C.json'},
  {'type': 'neu',
   'content': '星講\u3000的\u3000笑話\u3000再\u3000怎麼\u3000好笑\u3000永遠\u3000紅\u3000不\u3000了\u3000永遠\u3000被\u3000轉台',
   'board': 'Boy-Girl',
   'ord': 21,
   'src': '20191021_0204_M.1571594694.A.D9C.json'},
  {'type': 'pos',
   'content': '那\u3000同事\u3000說\u3000的\u3000話\u3000不\u3000是\u3000同事\u3000的\u3000心聲\u3000，\u3000其實\u3000是\u3000你\u3000男友\u3000的',
   'board': 'Boy-Girl',
   'ord': 719,
   'src': '20191021_0208_M.1571594908.A.C84.json'}],
 '2019-03-13': [{'type': 'pos',
   'content': '我\u3000也\u3000常常\u3000被\u3000叫\u3000去\u3000幫忙\u3000買\u3000東西\u3000跟\u3000倒水\u3000，\u3000也\u3000會\u3000不爽\u3000(\u3000因為\u3000我\u3000也\u3000懶\u3000呀\u3000)',
   'board': 'Boy-Girl',
   'ord': 262,
   'src': '20190313_2118_

## Possible Usage

In [ ]:
import os
from pttnet.graph import Node

node_ids = {f[:-11] for f in os.listdir("data/network/nodes") if f.endswith('.json')}
nodes = {id_: Node(id_, from_disk="data/network/nodes") for id_ in node_ids}


In [ ]:
#[n for k, n in nodes.items() if len(n.corpus) > 2]
nodes.get("Nekone").getCorpusStats()